# Fetch historical data

Python script to fetch historical data from binance using ccxt

In [5]:
!pip install openpyxl

  Created wheel for openpyxl: filename=openpyxl-3.0.0-py2.py3-none-any.whl size=241193 sha256=8bc5909533679c0c1e54d9164a2c38f6848177764b3e3c9e8a5c742ebe9a69e7
  Stored in directory: C:\Users\codeninja\AppData\Local\pip\Cache\wheels\34\ee\6c\1279f7b70ea72432c2cef15dd3d915477cb3771d1618f6b8ef
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-cp37-none-any.whl size=8922 sha256=89a5e1f98f64484b70b26d922b0062a6fe4d338d98bb5c2437ea6a4c2a21d38f
  Stored in directory: C:\Users\codeninja\AppData\Local\pip\Cache\wheels\2a\77\35\0da0965a057698121fc7d8c5a7a9955cdbfb3cc4e2423cad39
Successfully built openpyxl et-xmlfile


In [1]:
# -*- coding: utf-8 -*-

import os
from pathlib import Path

import sys
import csv

# -----------------------------------------------------------------------------

root = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath(''))))
sys.path.append(root + '/python')

import ccxt  # noqa: E402


# -----------------------------------------------------------------------------

def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    earliest_timestamp = exchange.milliseconds()
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    all_ohlcv = []
    while True:
        fetch_since = earliest_timestamp - timedelta
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        # if we have reached the beginning of history
        if ohlcv[0][0] >= earliest_timestamp:
            break
        earliest_timestamp = ohlcv[0][0]
        all_ohlcv = ohlcv + all_ohlcv
        print(len(all_ohlcv), symbol, 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        # if we have reached the checkpoint
        if fetch_since < since:
            break
    return all_ohlcv


def write_to_csv(filename, exchange, data):
    p = Path("./data/raw/", str(exchange))
    p.mkdir(parents=True, exist_ok=True)
    full_path = p / str(filename)
    with Path(full_path).open('w+', newline='') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)


def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, exchange, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)


# -----------------------------------------------------------------------------

scrape_candles_to_csv('btc_usdt_1m.csv', 'binance', 3, 'BTC/USDT', '1m', '2019-01-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/eth_btc_1m.csv', 'binance', 3, 'ETH/BTC', '1m', '2018-01-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/ltc_btc_1m.csv', 'binance', 3, 'LTC/BTC', '1m', '2018-01-01T00:00:00Z', 1000)
# scrape_candles_to_csv('./data/raw/binance/xlm_btc_1m.csv', 'binance', 3, 'XLM/BTC', '1m', '2018-01-01T00:00:00Z', 1000)

1000 BTC/USDT candles in total from 2019-10-22T02:13:00.000Z to 2019-10-22T18:52:00.000Z
2000 BTC/USDT candles in total from 2019-10-21T09:33:00.000Z to 2019-10-22T18:52:00.000Z
3000 BTC/USDT candles in total from 2019-10-20T16:53:00.000Z to 2019-10-22T18:52:00.000Z
4000 BTC/USDT candles in total from 2019-10-20T00:13:00.000Z to 2019-10-22T18:52:00.000Z
5000 BTC/USDT candles in total from 2019-10-19T07:33:00.000Z to 2019-10-22T18:52:00.000Z
6000 BTC/USDT candles in total from 2019-10-18T14:53:00.000Z to 2019-10-22T18:52:00.000Z
7000 BTC/USDT candles in total from 2019-10-17T22:13:00.000Z to 2019-10-22T18:52:00.000Z
8000 BTC/USDT candles in total from 2019-10-17T05:33:00.000Z to 2019-10-22T18:52:00.000Z
9000 BTC/USDT candles in total from 2019-10-16T12:53:00.000Z to 2019-10-22T18:52:00.000Z
10000 BTC/USDT candles in total from 2019-10-15T20:13:00.000Z to 2019-10-22T18:52:00.000Z
11000 BTC/USDT candles in total from 2019-10-15T03:33:00.000Z to 2019-10-22T18:52:00.000Z
12000 BTC/USDT cand

93000 BTC/USDT candles in total from 2019-08-19T04:53:00.000Z to 2019-10-22T18:52:00.000Z
94000 BTC/USDT candles in total from 2019-08-18T12:13:00.000Z to 2019-10-22T18:52:00.000Z
95000 BTC/USDT candles in total from 2019-08-17T19:33:00.000Z to 2019-10-22T18:52:00.000Z
96000 BTC/USDT candles in total from 2019-08-17T02:53:00.000Z to 2019-10-22T18:52:00.000Z
97000 BTC/USDT candles in total from 2019-08-16T10:13:00.000Z to 2019-10-22T18:52:00.000Z
98000 BTC/USDT candles in total from 2019-08-15T17:33:00.000Z to 2019-10-22T18:52:00.000Z
99000 BTC/USDT candles in total from 2019-08-15T00:53:00.000Z to 2019-10-22T18:52:00.000Z
100000 BTC/USDT candles in total from 2019-08-14T08:13:00.000Z to 2019-10-22T18:52:00.000Z
101000 BTC/USDT candles in total from 2019-08-13T15:33:00.000Z to 2019-10-22T18:52:00.000Z
102000 BTC/USDT candles in total from 2019-08-12T22:53:00.000Z to 2019-10-22T18:52:00.000Z
103000 BTC/USDT candles in total from 2019-08-12T06:13:00.000Z to 2019-10-22T18:52:00.000Z
104000

184000 BTC/USDT candles in total from 2019-06-17T00:13:00.000Z to 2019-10-22T18:52:00.000Z
185000 BTC/USDT candles in total from 2019-06-16T07:33:00.000Z to 2019-10-22T18:52:00.000Z
186000 BTC/USDT candles in total from 2019-06-15T14:53:00.000Z to 2019-10-22T18:52:00.000Z
187000 BTC/USDT candles in total from 2019-06-14T22:13:00.000Z to 2019-10-22T18:52:00.000Z
188000 BTC/USDT candles in total from 2019-06-14T05:33:00.000Z to 2019-10-22T18:52:00.000Z
189000 BTC/USDT candles in total from 2019-06-13T12:53:00.000Z to 2019-10-22T18:52:00.000Z
190000 BTC/USDT candles in total from 2019-06-12T20:13:00.000Z to 2019-10-22T18:52:00.000Z
191000 BTC/USDT candles in total from 2019-06-12T03:33:00.000Z to 2019-10-22T18:52:00.000Z
192000 BTC/USDT candles in total from 2019-06-11T10:53:00.000Z to 2019-10-22T18:52:00.000Z
193000 BTC/USDT candles in total from 2019-06-10T18:13:00.000Z to 2019-10-22T18:52:00.000Z
194000 BTC/USDT candles in total from 2019-06-10T01:33:00.000Z to 2019-10-22T18:52:00.000Z

275000 BTC/USDT candles in total from 2019-04-14T23:40:00.000Z to 2019-10-22T18:52:00.000Z
276000 BTC/USDT candles in total from 2019-04-14T07:00:00.000Z to 2019-10-22T18:52:00.000Z
277000 BTC/USDT candles in total from 2019-04-13T14:20:00.000Z to 2019-10-22T18:52:00.000Z
278000 BTC/USDT candles in total from 2019-04-12T21:40:00.000Z to 2019-10-22T18:52:00.000Z
279000 BTC/USDT candles in total from 2019-04-12T05:00:00.000Z to 2019-10-22T18:52:00.000Z
280000 BTC/USDT candles in total from 2019-04-11T12:20:00.000Z to 2019-10-22T18:52:00.000Z
281000 BTC/USDT candles in total from 2019-04-10T19:40:00.000Z to 2019-10-22T18:52:00.000Z
282000 BTC/USDT candles in total from 2019-04-10T03:00:00.000Z to 2019-10-22T18:52:00.000Z
283000 BTC/USDT candles in total from 2019-04-09T10:20:00.000Z to 2019-10-22T18:52:00.000Z
284000 BTC/USDT candles in total from 2019-04-08T17:40:00.000Z to 2019-10-22T18:52:00.000Z
285000 BTC/USDT candles in total from 2019-04-08T01:00:00.000Z to 2019-10-22T18:52:00.000Z

366000 BTC/USDT candles in total from 2019-02-10T19:00:00.000Z to 2019-10-22T18:52:00.000Z
367000 BTC/USDT candles in total from 2019-02-10T02:20:00.000Z to 2019-10-22T18:52:00.000Z
368000 BTC/USDT candles in total from 2019-02-09T09:40:00.000Z to 2019-10-22T18:52:00.000Z
369000 BTC/USDT candles in total from 2019-02-08T17:00:00.000Z to 2019-10-22T18:52:00.000Z
370000 BTC/USDT candles in total from 2019-02-08T00:20:00.000Z to 2019-10-22T18:52:00.000Z
371000 BTC/USDT candles in total from 2019-02-07T07:40:00.000Z to 2019-10-22T18:52:00.000Z
372000 BTC/USDT candles in total from 2019-02-06T15:00:00.000Z to 2019-10-22T18:52:00.000Z
373000 BTC/USDT candles in total from 2019-02-05T22:20:00.000Z to 2019-10-22T18:52:00.000Z
374000 BTC/USDT candles in total from 2019-02-05T05:40:00.000Z to 2019-10-22T18:52:00.000Z
375000 BTC/USDT candles in total from 2019-02-04T13:00:00.000Z to 2019-10-22T18:52:00.000Z
376000 BTC/USDT candles in total from 2019-02-03T20:20:00.000Z to 2019-10-22T18:52:00.000Z